In [46]:
from music21 import converter, instrument, note, chord, stream
import glob
import numpy as np
from keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, Lambda
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [47]:
nocturne = converter.parse("music/Chopin_Etuden_4.mid")

parts = instrument.partitionByInstrument(nocturne)

notes_to_parse = parts.parts[0].recurse()  # 対象のパートを一つに絞る

string_nocturne_notes = []

for element in notes_to_parse:
    if isinstance(element, note.Note):
      string_nocturne_notes.append(str(element.pitch))
    elif isinstance(element, chord.Chord):
      string_nocturne_notes.append('.'.join(str(n) for n in element.normalOrder))

In [48]:
string_nocturne_notes

['G#5',
 'G#4',
 '8',
 'F#5',
 'E5',
 'E-5',
 'C#5',
 'C#6',
 '1',
 'E-5',
 'C5',
 'C#5',
 'E-5',
 '6.9',
 'E5',
 'C#5',
 'E-5',
 'E5',
 '4.8',
 'F#5',
 'E-5',
 'E5',
 'F#5',
 '3.6',
 'G#5',
 'E5',
 'F#5',
 'G#5',
 '1.4',
 'A5',
 'G#5',
 'C#6',
 'F#5',
 '6.9.1',
 'G#5',
 'F#5',
 'C#6',
 'E5',
 '1.4.8',
 'F#5',
 'E5',
 'C#6',
 'E-5',
 '1.3.6',
 'E5',
 'E-5',
 'C#6',
 '1.4.8',
 'C#5',
 'G#5',
 'E6',
 'A5',
 'C5',
 '6.8.0',
 'F#5',
 'E-6',
 'G#5',
 'G#4',
 '1.4',
 'E5',
 'C#6',
 'F#5',
 'G#4',
 'G#3',
 'G#2',
 'E-5',
 'C6',
 'E5',
 'C#6',
 'C#5',
 '1',
 'A4',
 '9',
 'A5',
 'G#4',
 '8',
 'G#5',
 'G4',
 '7',
 'G5',
 'G#4',
 '8',
 'G#5',
 'C#4',
 '1',
 'C#5',
 'E-5',
 '7.10.1.3',
 '3',
 'C#3',
 'B2',
 'B-2',
 'E-5',
 '8.11.3',
 'G#2',
 'B-2',
 'G2',
 'G#2',
 '1.3.7',
 'B-2',
 'B2',
 'G#2',
 'B-2',
 '8.11.3',
 'B2',
 'C#3',
 'B-2',
 'B2',
 '4.8.10',
 'C#3',
 'E-3',
 'B2',
 'C#3',
 '8.11.3',
 'E-3',
 'E3',
 'E-3',
 'G#3',
 '1.4.8',
 'C#3',
 'E-3',
 'C#3',
 'G#3',
 '3.8',
 'B2',
 'C#3',
 'B2',


In [49]:
notenames = sorted(set(string_nocturne_notes)) #音名
notenames

['0',
 '0.3',
 '0.3.5',
 '0.3.6',
 '0.3.6.8',
 '0.6',
 '1',
 '1.3',
 '1.3.6',
 '1.3.7',
 '1.4',
 '1.4.8',
 '1.5',
 '1.7',
 '10',
 '10.1',
 '10.1.3',
 '10.1.4',
 '10.1.5',
 '10.11',
 '10.3',
 '11',
 '11.2',
 '11.2.5',
 '11.3',
 '11.3.4.5',
 '11.4',
 '2',
 '3',
 '3.6',
 '3.6.10',
 '3.6.8',
 '3.6.8.9',
 '3.7.10',
 '3.8',
 '4',
 '4.10',
 '4.7.10',
 '4.8',
 '4.8.10',
 '4.9',
 '5',
 '5.11',
 '5.8.11',
 '5.9',
 '5.9.0',
 '6',
 '6.11',
 '6.8',
 '6.8.0',
 '6.9',
 '6.9.1',
 '7',
 '7.10.1',
 '7.10.1.3',
 '7.11',
 '7.8',
 '8',
 '8.0',
 '8.1',
 '8.11',
 '8.11.1',
 '8.11.2',
 '8.11.3',
 '9',
 '9.0.3',
 '9.0.4',
 '9.1.3',
 '9.1.4',
 '9.11',
 '9.2',
 'A1',
 'A2',
 'A3',
 'A4',
 'A5',
 'A6',
 'B-2',
 'B-3',
 'B-4',
 'B-5',
 'B-6',
 'B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'C#2',
 'C#3',
 'C#4',
 'C#5',
 'C#6',
 'C#7',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'D3',
 'D4',
 'D5',
 'D6',
 'E-2',
 'E-3',
 'E-4',
 'E-5',
 'E-6',
 'E2',
 'E3',
 'E4',
 'E5',
 'E6',
 'E7',
 'F#2',
 'F#3',
 'F#4',
 'F#5',
 'F#6',
 'F2',
 

In [50]:
note2int = dict((string_note, number) for number, string_note in enumerate(notenames))
note2int

{'0': 0,
 '0.3': 1,
 '0.3.5': 2,
 '0.3.6': 3,
 '0.3.6.8': 4,
 '0.6': 5,
 '1': 6,
 '1.3': 7,
 '1.3.6': 8,
 '1.3.7': 9,
 '1.4': 10,
 '1.4.8': 11,
 '1.5': 12,
 '1.7': 13,
 '10': 14,
 '10.1': 15,
 '10.1.3': 16,
 '10.1.4': 17,
 '10.1.5': 18,
 '10.11': 19,
 '10.3': 20,
 '11': 21,
 '11.2': 22,
 '11.2.5': 23,
 '11.3': 24,
 '11.3.4.5': 25,
 '11.4': 26,
 '2': 27,
 '3': 28,
 '3.6': 29,
 '3.6.10': 30,
 '3.6.8': 31,
 '3.6.8.9': 32,
 '3.7.10': 33,
 '3.8': 34,
 '4': 35,
 '4.10': 36,
 '4.7.10': 37,
 '4.8': 38,
 '4.8.10': 39,
 '4.9': 40,
 '5': 41,
 '5.11': 42,
 '5.8.11': 43,
 '5.9': 44,
 '5.9.0': 45,
 '6': 46,
 '6.11': 47,
 '6.8': 48,
 '6.8.0': 49,
 '6.9': 50,
 '6.9.1': 51,
 '7': 52,
 '7.10.1': 53,
 '7.10.1.3': 54,
 '7.11': 55,
 '7.8': 56,
 '8': 57,
 '8.0': 58,
 '8.1': 59,
 '8.11': 60,
 '8.11.1': 61,
 '8.11.2': 62,
 '8.11.3': 63,
 '9': 64,
 '9.0.3': 65,
 '9.0.4': 66,
 '9.1.3': 67,
 '9.1.4': 68,
 '9.11': 69,
 '9.2': 70,
 'A1': 71,
 'A2': 72,
 'A3': 73,
 'A4': 74,
 'A5': 75,
 'A6': 76,
 'B-2': 77,
 'B-3'

In [51]:
numerical_nocturne_notes = []
for string_note in string_nocturne_notes:
    numerical_nocturne_notes.append(note2int[string_note])
    
numerical_nocturne_notes

[127,
 126,
 57,
 116,
 110,
 105,
 90,
 91,
 6,
 105,
 95,
 90,
 105,
 50,
 110,
 90,
 105,
 110,
 38,
 116,
 105,
 110,
 116,
 29,
 127,
 110,
 116,
 127,
 10,
 75,
 127,
 91,
 116,
 51,
 127,
 116,
 91,
 110,
 11,
 116,
 110,
 91,
 105,
 8,
 110,
 105,
 91,
 11,
 90,
 127,
 111,
 75,
 95,
 49,
 116,
 106,
 127,
 126,
 10,
 110,
 91,
 116,
 126,
 125,
 124,
 105,
 96,
 110,
 91,
 90,
 6,
 74,
 64,
 75,
 126,
 57,
 127,
 131,
 52,
 132,
 126,
 57,
 127,
 89,
 6,
 90,
 105,
 54,
 28,
 88,
 83,
 77,
 105,
 63,
 124,
 77,
 129,
 124,
 9,
 77,
 83,
 124,
 77,
 63,
 83,
 88,
 77,
 83,
 39,
 88,
 103,
 83,
 88,
 63,
 103,
 108,
 103,
 125,
 11,
 88,
 103,
 88,
 125,
 34,
 83,
 88,
 83,
 125,
 16,
 77,
 83,
 77,
 130,
 63,
 124,
 108,
 84,
 103,
 106,
 80,
 105,
 129,
 108,
 78,
 103,
 86,
 127,
 124,
 108,
 84,
 103,
 105,
 79,
 104,
 129,
 108,
 78,
 103,
 85,
 126,
 104,
 57,
 109,
 89,
 131,
 104,
 84,
 126,
 131,
 109,
 79,
 115,
 104,
 85,
 74,
 115,
 95,
 126,
 109,
 90,
 85,
 31,
 10

In [52]:
sequence_length = 100 #  1つの入力の長さ

network_input = []
network_output = []

for i in range(0, len(numerical_nocturne_notes) - sequence_length, 1):
    network_input.append(numerical_nocturne_notes[i:i + sequence_length])
    network_output.append(numerical_nocturne_notes[i + sequence_length])

In [53]:
#network_inputはTensorflowで扱えるようにnumpyにしておきます。
network_input = np.reshape(network_input, (-1, sequence_length, 1))

In [54]:
network_input.shape, network_input

((1793, 100, 1),
 array([[[127],
         [126],
         [ 57],
         ...,
         [124],
         [  9],
         [ 77]],
 
        [[126],
         [ 57],
         [116],
         ...,
         [  9],
         [ 77],
         [ 83]],
 
        [[ 57],
         [116],
         [110],
         ...,
         [ 77],
         [ 83],
         [124]],
 
        ...,
 
        [[ 90],
         [ 89],
         [ 80],
         ...,
         [ 11],
         [ 11],
         [  6]],
 
        [[ 89],
         [ 80],
         [124],
         ...,
         [ 11],
         [  6],
         [  6]],
 
        [[ 80],
         [124],
         [ 95],
         ...,
         [  6],
         [  6],
         [  6]]]))

In [55]:
network_output = np_utils.to_categorical(network_output)

In [56]:
network_output.shape, network_output

((1793, 134),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32))

In [57]:
#midiファイルの数だけ作成
sequence_length = 100 #  1つの入力の長さ
string_flatten_network_input = []
string_network_output = []

for music in glob.glob("music/Chopin_Etuden_4.mid"):
    string_notes = []
    print("parsing " + music)
    midi = converter.parse(music)
    
    note_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    
    if parts:
        note_to_parse = parts.parts[0].recurse()
    else:
        note_to_parse = music.flat.notes
        
    for element in note_to_parse:
        if isinstance(element, note.Note):
            string_notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            string_notes.append('.'.join(str(n) for n in element.normalOrder))
            
    for i in range(0, len(string_notes) - sequence_length, 1):
        for string_note in string_notes[i:i+sequence_length]:
            string_flatten_network_input.append(string_note)
            
        string_network_output.append(string_notes[i+sequence_length])

parsing music/Chopin_Etuden_4.mid


In [58]:
sequence_length = 100 #  1つの入力の長さ
string_flatten_network_input = []
string_network_output = []

for music in glob.glob("music/Chopin_Etuden_op10-12.mid"):
    string_notes = []
    print("parsing " + music)
    midi = converter.parse(music)
    
    note_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    
    if parts:
        note_to_parse = parts.parts[0].recurse()
    else:
        note_to_parse = music.flat.notes
        
    for element in note_to_parse:
        if isinstance(element, note.Note):
            string_notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            string_notes.append('.'.join(str(n) for n in element.normalOrder))
            
    for i in range(0, len(string_notes) - sequence_length, 1):
        for string_note in string_notes[i:i+sequence_length]:
            string_flatten_network_input.append(string_note)
            
        string_network_output.append(string_notes[i+sequence_length])

parsing music/Chopin_Etuden_op10-12.mid


In [59]:
sequence_length = 100 #  1つの入力の長さ
string_flatten_network_input = []
string_network_output = []

for music in glob.glob("music/Chopin_Grand_Valse_Brillante_1.mid"):
    string_notes = []
    print("parsing " + music)
    midi = converter.parse(music)
    
    note_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    
    if parts:
        note_to_parse = parts.parts[0].recurse()
    else:
        note_to_parse = music.flat.notes
        
    for element in note_to_parse:
        if isinstance(element, note.Note):
            string_notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            string_notes.append('.'.join(str(n) for n in element.normalOrder))
            
    for i in range(0, len(string_notes) - sequence_length, 1):
        for string_note in string_notes[i:i+sequence_length]:
            string_flatten_network_input.append(string_note)
            
        string_network_output.append(string_notes[i+sequence_length])

parsing music/Chopin_Grand_Valse_Brillante_1.mid


In [60]:
sequence_length = 100 #  1つの入力の長さ
string_flatten_network_input = []
string_network_output = []

for music in glob.glob("music/Chopin_Valse_2.mid"):
    string_notes = []
    print("parsing " + music)
    midi = converter.parse(music)
    
    note_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    
    if parts:
        note_to_parse = parts.parts[0].recurse()
    else:
        note_to_parse = music.flat.notes
        
    for element in note_to_parse:
        if isinstance(element, note.Note):
            string_notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            string_notes.append('.'.join(str(n) for n in element.normalOrder))
            
    for i in range(0, len(string_notes) - sequence_length, 1):
        for string_note in string_notes[i:i+sequence_length]:
            string_flatten_network_input.append(string_note)
            
        string_network_output.append(string_notes[i+sequence_length])

parsing music/Chopin_Valse_2.mid


In [61]:
sequence_length = 100 #  1つの入力の長さ
string_flatten_network_input = []
string_network_output = []

for music in glob.glob("music/Chopin_Valse_14.mid"):
    string_notes = []
    print("parsing " + music)
    midi = converter.parse(music)
    
    note_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    
    if parts:
        note_to_parse = parts.parts[0].recurse()
    else:
        note_to_parse = music.flat.notes
        
    for element in note_to_parse:
        if isinstance(element, note.Note):
            string_notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            string_notes.append('.'.join(str(n) for n in element.normalOrder))
            
    for i in range(0, len(string_notes) - sequence_length, 1):
        for string_note in string_notes[i:i+sequence_length]:
            string_flatten_network_input.append(string_note)
            
        string_network_output.append(string_notes[i+sequence_length])

parsing music/Chopin_Valse_14.mid


In [62]:
sequence_length = 100 #  1つの入力の長さ
string_flatten_network_input = []
string_network_output = []

for music in glob.glob("music/CHOPV6.mid"):
    string_notes = []
    print("parsing " + music)
    midi = converter.parse(music)
    
    note_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    
    if parts:
        note_to_parse = parts.parts[0].recurse()
    else:
        note_to_parse = music.flat.notes
        
    for element in note_to_parse:
        if isinstance(element, note.Note):
            string_notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            string_notes.append('.'.join(str(n) for n in element.normalOrder))
            
    for i in range(0, len(string_notes) - sequence_length, 1):
        for string_note in string_notes[i:i+sequence_length]:
            string_flatten_network_input.append(string_note)
            
        string_network_output.append(string_notes[i+sequence_length])

parsing music/CHOPV6.mid


In [63]:
sequence_length = 100 #  1つの入力の長さ
string_flatten_network_input = []
string_network_output = []

for music in glob.glob("music/CHOVA31.mid"):
    string_notes = []
    print("parsing " + music)
    midi = converter.parse(music)
    
    note_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    
    if parts:
        note_to_parse = parts.parts[0].recurse()
    else:
        note_to_parse = music.flat.notes
        
    for element in note_to_parse:
        if isinstance(element, note.Note):
            string_notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            string_notes.append('.'.join(str(n) for n in element.normalOrder))
            
    for i in range(0, len(string_notes) - sequence_length, 1):
        for string_note in string_notes[i:i+sequence_length]:
            string_flatten_network_input.append(string_note)
            
        string_network_output.append(string_notes[i+sequence_length])

parsing music/CHOVA31.mid


In [64]:
len(string_flatten_network_input)

146800

In [65]:
len(string_network_output)

1468

In [66]:
notenames = sorted(set(string_flatten_network_input + string_network_output))
note2int = dict((string_note, number) for number, string_note in enumerate(notenames))

In [67]:
notenames

['0.4',
 '0.4.7',
 '1.4',
 '1.4.7',
 '10.2.5',
 '11.2',
 '11.2.4',
 '11.2.6',
 '11.3',
 '2.4',
 '2.4.8',
 '2.5',
 '2.5.8',
 '2.5.9',
 '2.6',
 '2.6.8',
 '2.8',
 '3.4',
 '3.8',
 '3.9',
 '4.5',
 '4.5.9',
 '4.7',
 '4.7.9',
 '4.8',
 '4.8.11',
 '4.9',
 '5.10',
 '5.11',
 '5.7',
 '5.7.11',
 '5.7.9',
 '5.8.11',
 '5.9',
 '6.11',
 '6.8',
 '6.9',
 '7.0',
 '7.9',
 '7.9.1',
 '8.11',
 '8.11.2',
 '8.9',
 '9.0',
 '9.0.4',
 '9.1',
 '9.1.4',
 '9.11',
 '9.11.3',
 '9.2',
 'A2',
 'A3',
 'A4',
 'A5',
 'B-3',
 'B-4',
 'B2',
 'B3',
 'B4',
 'C#3',
 'C#4',
 'C#5',
 'C#6',
 'C3',
 'C4',
 'C5',
 'C6',
 'D3',
 'D4',
 'D5',
 'D6',
 'E-3',
 'E-4',
 'E-5',
 'E2',
 'E3',
 'E4',
 'E5',
 'F#3',
 'F#4',
 'F#5',
 'F2',
 'F3',
 'F4',
 'F5',
 'G#2',
 'G#3',
 'G#4',
 'G#5',
 'G2',
 'G3',
 'G4',
 'G5']

In [68]:
network_input = []
for string_note in string_flatten_network_input:
    network_input.append(note2int[string_note])
    
network_input = np.reshape(network_input, (-1, sequence_length, 1))
#network_input = network_input / float(n_vocab)

In [69]:
network_input.shape, network_input

((1468, 100, 1),
 array([[[75],
         [50],
         [76],
         ...,
         [68],
         [57],
         [52]],
 
        [[50],
         [76],
         [64],
         ...,
         [57],
         [52],
         [57]],
 
        [[76],
         [64],
         [52],
         ...,
         [52],
         [57],
         [90]],
 
        ...,
 
        [[75],
         [82],
         [50],
         ...,
         [76],
         [16],
         [64]],
 
        [[82],
         [50],
         [57],
         ...,
         [16],
         [64],
         [57]],
 
        [[50],
         [57],
         [68],
         ...,
         [64],
         [57],
         [43]]]))

In [70]:
network_output = []
for string_note in string_network_output:
    network_output.append(note2int[string_note])

network_output = np_utils.to_categorical(network_output)

In [71]:
network_output.shape, network_output

((1468, 93),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32))

In [72]:
n_vocab = len(notenames)

In [73]:
model = Sequential()
model.add(Lambda((lambda x: x/n_vocab), input_shape=(network_input.shape[1], network_input.shape[2])))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))

In [74]:
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.0001)) # このlrがうまくいきました。

In [75]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda_1 (Lambda)           (None, 100, 1)            0         
                                                                 
 lstm_3 (LSTM)               (None, 100, 256)          264192    
                                                                 
 dropout_3 (Dropout)         (None, 100, 256)          0         
                                                                 
 lstm_4 (LSTM)               (None, 100, 256)          525312    
                                                                 
 dropout_4 (Dropout)         (None, 100, 256)          0         
                                                                 
 lstm_5 (LSTM)               (None, 256)               525312    
                                                                 
 dropout_5 (Dropout)         (None, 256)              

In [76]:
dir = "model_checkpoint/"
filepath = "{loss:.4f}-weights-improvement-{epoch:02d}.hdf5"

checkpoint = ModelCheckpoint(
    filepath = dir + filepath, monitor="loss", verbose=0, save_best_only=True, mode='min', period=10
)

In [77]:
callbacks_list = [checkpoint]

In [78]:
#学習開始
#model.fit(network_input, network_output, epochs=500, batch_size=64, callbacks=callbacks_list)

In [79]:
music_length = 250
start = np.random.randint(0, len(network_input)-1)
pattern = network_input[start]

In [80]:
pattern

array([[ 5],
       [58],
       [52],
       [82],
       [43],
       [71],
       [76],
       [75],
       [52],
       [43],
       [58],
       [65],
       [69],
       [77],
       [84],
       [70],
       [66],
       [77],
       [65],
       [53],
       [84],
       [74],
       [52],
       [83],
       [10],
       [69],
       [65],
       [10],
       [76],
       [64],
       [52],
       [50],
       [26],
       [76],
       [77],
       [50],
       [52],
       [ 0],
       [ 0],
       [58],
       [56],
       [52],
       [ 9],
       [87],
       [ 9],
       [84],
       [56],
       [58],
       [12],
       [12],
       [65],
       [63],
       [58],
       [33],
       [52],
       [33],
       [53],
       [59],
       [31],
       [92],
       [92],
       [23],
       [84],
       [77],
       [92],
       [67],
       [84],
       [52],
       [21],
       [58],
       [65],
       [13],
       [69],
       [77],
       [82],
       [69],
       [68],

In [81]:
int2note = dict((number, string_note) for number, string_note in enumerate(notenames))
int2note

{0: '0.4',
 1: '0.4.7',
 2: '1.4',
 3: '1.4.7',
 4: '10.2.5',
 5: '11.2',
 6: '11.2.4',
 7: '11.2.6',
 8: '11.3',
 9: '2.4',
 10: '2.4.8',
 11: '2.5',
 12: '2.5.8',
 13: '2.5.9',
 14: '2.6',
 15: '2.6.8',
 16: '2.8',
 17: '3.4',
 18: '3.8',
 19: '3.9',
 20: '4.5',
 21: '4.5.9',
 22: '4.7',
 23: '4.7.9',
 24: '4.8',
 25: '4.8.11',
 26: '4.9',
 27: '5.10',
 28: '5.11',
 29: '5.7',
 30: '5.7.11',
 31: '5.7.9',
 32: '5.8.11',
 33: '5.9',
 34: '6.11',
 35: '6.8',
 36: '6.9',
 37: '7.0',
 38: '7.9',
 39: '7.9.1',
 40: '8.11',
 41: '8.11.2',
 42: '8.9',
 43: '9.0',
 44: '9.0.4',
 45: '9.1',
 46: '9.1.4',
 47: '9.11',
 48: '9.11.3',
 49: '9.2',
 50: 'A2',
 51: 'A3',
 52: 'A4',
 53: 'A5',
 54: 'B-3',
 55: 'B-4',
 56: 'B2',
 57: 'B3',
 58: 'B4',
 59: 'C#3',
 60: 'C#4',
 61: 'C#5',
 62: 'C#6',
 63: 'C3',
 64: 'C4',
 65: 'C5',
 66: 'C6',
 67: 'D3',
 68: 'D4',
 69: 'D5',
 70: 'D6',
 71: 'E-3',
 72: 'E-4',
 73: 'E-5',
 74: 'E2',
 75: 'E3',
 76: 'E4',
 77: 'E5',
 78: 'F#3',
 79: 'F#4',
 80: 'F#5',
 8

In [82]:
numerical_prediction_output = []

for note_index in range(music_length):
  prediction_input = np.reshape(pattern, (1, len(pattern), 1))

  prediction = model.predict(prediction_input, verbose=0)
  
  numerical_note = np.argmax(prediction)
  numerical_prediction_output.append(numerical_note)
  
  #pattern = np.append(pattern, numerical_note/float(n_vocab))
  pattern = np.append(pattern, numerical_note)
  pattern = pattern[1:len(pattern)]

In [83]:
string_prediction_output = []

for i in range(music_length):
    string_prediction_output.append(int2note[numerical_prediction_output[i]])

In [84]:
offset = 0
prediction_output = []

for string_note in string_prediction_output:
  if ('.' in string_note) or string_note.isdigit():
    notes_in_chord = string_note.split('.')
    notes = []
    for current_note in notes_in_chord:
      new_note = note.Note(int(current_note))
      new_note.storedInstrument = instrument.Piano()
      notes.append(new_note)
    new_chord = chord.Chord(notes)
    new_chord.offset = offset
    prediction_output.append(new_chord)
  else:
    new_note = note.Note(string_note)
    new_note.offset = offset
    new_note.storedInstrument = instrument.Piano()
    prediction_output.append(new_note)
    
  offset += 0.5

In [85]:
prediction_output

[<music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.note.N

In [86]:
midi_stream = stream.Stream(prediction_output)
midi_stream.write('midi', fp='becoming_music/test_output.mid')

'becoming_music/test_output.mid'